## CPL: 2-Stage DTR

In [1]:
import pandas as pd
DTR_data = pd.read_csv('mimic3_DTR.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'mimic3_DTR.csv'

In [2]:
DTR_data.head()

,icustayid_1,Glucose_1,paO2_1,PaO2_FiO2_1,IV_Input_1,SOFA_1,Glucose_2,paO2_2,PaO2_FiO2_2,IV_Input_2,SOFA_2,Died_within_48H
0,31005.0,116.833333,86.0,172.000000,1.0,9.0,125.666667,182.000000,364.000000,1.0,5.0,1.0
1,10989.0,120.000000,85.0,170.000000,1.0,5.0,154.000000,81.857143,163.714286,1.0,7.0,1.0
2,4132.0,123.200000,133.0,266.000000,1.0,8.0,126.400000,47.000000,94.000000,1.0,7.0,-1.0
3,37528.0,168.500000,62.6,260.833333,0.0,3.0,227.000000,66.666667,277.777778,1.0,4.0,-1.0
4,86428.0,115.000000,73.0,146.000000,1.0,7.0,115.000000,104.333333,208.666667,1.0,8.0,1.0


In [3]:
import os
os.getcwd()
os.chdir('/nas/longleaf/home/lge/CausalDM')
from causaldm.learners import QLearning

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


## Binary IV_Input

As an example, we use the **Q-learning** algorithm to optimize and evaluate policy. We begin by training the Q models on the observed data.  By assuming linear regression models and using the code in the following block, we have that A = B.
\begin{align}
Q_1(s,a_1) = &.7725+.0016*\textrm{Glucose}_1-.0015*\textrm{paO2}_1+.0003*\textrm{PaO2_FiO2}_1-.0089*\textrm{SOFA}_1+\\
                    &I(a_1=1)*\{-.1308-.0021*\textrm{Glucose}_1-.0054*\textrm{paO2}_1+.0021*\textrm{PaO2_FiO2}_1+.0973*\textrm{SOFA}_1\},\\
Q_2(s,a_2) = &2.4362+.0008*\textrm{Glucose}_1-.0008*\textrm{paO2}_1+.0004*\textrm{PaO2_FiO2}_1+.0363*\textrm{SOFA}_1+\\
                    &.0022*\textrm{Glucose}_2-.0144*\textrm{paO2}_2-.0003*\textrm{PaO2_FiO2}_2-.1306*\textrm{SOFA}_2+\\
                    &I(a_2=1)*\{-1.5796-.0044*\textrm{Glucose}_2+.0145*\textrm{paO2}_2+.0007*\textrm{PaO2_FiO2}_2+.0649*\textrm{SOFA}_2\}.
\end{align}



In [4]:
DTR_data.rename(columns = {'Died_within_48H':'R',
                            'Glucose_1':'S1_1', 'Glucose_2':'S1_2',
                            'paO2_1':'S2_1', 'paO2_2':'S2_2',
                            'PaO2_FiO2_1':'S3_1', 'PaO2_FiO2_2':'S3_2',
                            'SOFA_1':'S4_1', 'SOFA_2':'S4_2',
                            'IV_Input_1':'A1','IV_Input_2':'A2'}, inplace = True)
R = DTR_data['R'] #lower the better
S = DTR_data[['S1_1','S1_2','S2_1','S2_2','S3_1','S3_2','S4_1','S4_2']]
A = DTR_data[['A1','A2']]
# initialize the learner
QLearn = QLearning.QLearning()
# specify the model you would like to use
model_info = [{"model": "R~S1_1+S2_1+S3_1+S4_1+A1+S1_1*A1+S2_1*A1+S3_1*A1+S4_1*A1",
              'action_space':{'A1':[0,1]}},
             {"model": "R~S1_1+S2_1+S3_1+S4_1+S1_2+S2_2+S3_2+S4_2+A2+S1_2*A2+S2_2*A2+S3_2*A2+S4_2*A2",
              'action_space':{'A2':[0,1]}}]
# train the policy
QLearn.train(S, A, R, model_info, T=2, mimic3_clip = True)
# get the summary of the fitted Q models using the following code
#print("fitted model Q0:",QLearn.fitted_model[0].summary())
#print("fitted model Q1:",QLearn.fitted_model[1].summary())

{1: <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7fe8e45dd3a0>,
 0: <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x7fe8b7cb6400>}

**Policy Optimization:** Based on the fitted Q models, an optimal treatment regime can be defined as
- At stage 1:
    1. We would recommend $A=0$ (IV_Input = 0) if $-.0021*\textrm{Glucose}_1-.0054*\textrm{paO2}_1+.0021*\textrm{PaO2_FiO2}_1+.0973*\textrm{SOFA}_1<.1308$
    2. Else, we would recommend $A=1$ (IV_Input = 1).
- At stage 2:
    1. We would recommend $A=0$ (IV_Input = 0) if $-.0044*\textrm{Glucose}_2+.0145*\textrm{paO2}_2+.0007*\textrm{PaO2_FiO2}_2+.0649*\textrm{SOFA}_2<1.5796$
    2. Else, we would recommend $A=1$ (IV_Input = 1).
    
In the block that follows, we apply the estimated optimal regime to individuals in the observed data and estimate the corresponding policy value. Following the estimated optimal policy, 
- 20 patients are recommended to be treated with IV input at both stages, 
- nine patients are recommended not to be treated at both stages, 
- 19 patients are recommended to be treated with IV input at the first stage and not to be treated at the second stage,
- seven patients are recommended not to be treated at the first stage but to be treated with IV input at the second stage.

The estimated value of the estimated optimal policy is **.9357**.

In [5]:
#4. recommend action
opt_d = QLearn.recommend_action(S).value_counts()
#5. get the estimated value of the optimal regime
V_hat = QLearn.predict_value(S)
print("opt_d:",opt_d)
print("opt value:",V_hat)

opt_d: A2  A1
1   1     20
0   1     19
    0      9
1   0      7
dtype: int64
opt value: 0.9357210192715425


**Policy Evaluation:** Now, we are interested in evaluating a fixed policy that never gives IV input to patients. Under the specified model, the estimated value of the fixed policy that always does not give IV input is .4107.

In [6]:
#specify the fixed regime to be tested
# For example, regime d = 1 for all subjects at all decision points\
N=len(S)
# !! IMPORTANT: INDEX SHOULD BE THE SAME AS THAT OF THE S,R,A
regime = pd.DataFrame({'A1':[0]*N,
                      'A2':[0]*N}).set_index(S.index)
#evaluate the regime
QLearn = QLearning.QLearning()
QLearn.train(S, A, R, model_info, T=2, regime = regime, evaluate = True, mimic3_clip = True)
QLearn.predict_value(S)

0.4107373306703053